# Setup

In [ ]:
#prevent random disconnects
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

!pip install anvil-uplink

In [ ]:
#@title Install or update DeepFaceLab from Github { vertical-output: true }
#Mode = "install" #@param ["install", "update"]


pretrain_link = "https://github.com/chervonij/DFL-Colab/releases/download/"
pretrain_link = pretrain_link+"pretrain-CelebA/pretrain_CelebA.zip"

m = "install"

from pathlib import Path
if (m == "install"):
  !git clone https://github.com/iperov/DeepFaceLab.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/DeepFaceLab
  !git pull

!pip uninstall -y tensorflow
!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0

if not Path("/content/pretrain").exists():
  print("Downloading Pretrain faceset ... ")
  !wget -q --no-check-certificate -r $pretrain_link -O /content/pretrain_faceset.zip
  !mkdir /content/pretrain
  !unzip -q /content/pretrain_faceset.zip -d /content/pretrain/
  !rm /content/pretrain_faceset.zip

if not Path("/content/pretrain_Q96").exists():
  print("Downloading Q96 pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/pretrain_Q96.zip
  !mkdir /content/pretrain_Q96
  !unzip -q /content/pretrain_Q96.zip -d /content/pretrain_Q96/
  !rm /content/pretrain_Q96.zip

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")

In [ ]:
#download updated code files from gDrive

import shutil
!gdown --id 16eKsWHCH35NzHi_lqr95wtlxyudwL_v7
!gdown --id 1PImbxTmD7C2kywqY8HG5j9mO4RNVjZUU

shutil.move("/content/ModelBase.py", "/content/DeepFaceLab/models/ModelBase.py")
shutil.move("/content/Model.py", "/content/DeepFaceLab/models/Model_SAEHD/Model.py")

!gdown --id 1JYXyQreln8n3eQjT9Fld2vLBzIdg9qJW
!gdown --id 1bavD-FFdtOkXfiU2n03kEtYTLlAuarPz

shutil.move("/content/Merger.py", "/content/DeepFaceLab/mainscripts/Merger.py")
shutil.move("/content/MergerConfig.py", "/content/DeepFaceLab/merger/MergerConfig.py")

# Face Recognition

In [ ]:
#make directory of known faces
!mkdir known
!wget https://www.princeton.edu/sites/default/files/styles/scale_1440/public/images/2021/04/Trevor-Noah_CAA-Speakers_Photo-Credit-Gavin-Bond.jpg?itok=ilEPaXVg -O known/trevor.jpg
!wget https://www.biography.com/.image/t_share/MTczNjEwODI2NTg5MDg3MTI0/michelle-obama-gettyimages-85246899.jpg -O known/michelle.jpg
!wget https://www.thenews.com.pk/assets/uploads/updates/2021-04-17/821680_7280106_natalie-portman_updates.jpg -O known/natalie.jpg
!wget https://www.biography.com/.image/t_share/MTE4MDAzNDEwNzg2ODEzNDU0/jimmy-fallon-11818465-1-402.jpg -O known/jimmy.jpg
!wget https://media.gettyimages.com/photos/indian-bollywood-actress-kareena-kapoor-khan-looks-on-during-the-of-picture-id814885222?s=612x612 -O known/karina.jpg
!wget https://www.24newshd.tv/digital_images/large/2020-05-08/hania-amir-teases-fahad-mustafa-with-her-new-tik-tok-video-1588934215-4295.jpg -O known/hania.jpg
!wget https://www.samaa.tv/wp-content/uploads/2019/11/hamza-ali-abbasi-1.jpg -O known/hamza.jpg
!wget https://pbs.twimg.com/profile_images/1271836263268655106/6sWADZ0m.jpg -O known/ahad.jpg
!wget https://pbs.twimg.com/media/C0V7W29WIAASlIF.jpg -O known/ali.jpg


#install face recognition library
!pip install face_recognition

In [ ]:
import face_recognition   ##Detecting and recogniting faces
import cv2     ## intracting with images
import os       ## For Reading the file name
import shutil
from google.colab.patches import cv2_imshow ## we are importing cv2_imshow from google.colab.patches because google colab doesn't support cv2.imshow() funciton.

#function that can be used to read and reshape the images
def read_img(path):
   img = cv2.imread(path) ## reading image
   (h,w) = img.shape[:2]  ## fetching height and width
   width = 500            ## hard coding width
   ratio = width / float(w) ## preparing a ration for height
   height = int(h * ratio)  ## generating new height
   return cv2.resize(img,(width,height)) ##return the reshaped image

!mkdir face2

#The Face_recognition library recognizes faces by generating different-different encodings
#for each known face and unknown face and then both the encodings get compared and 
#whichever encoding is matched it shows the label name as output.
#Preparing encodings for Known Faces
known_encodings = [] 
known_names = []
known_dir = 'known' ##Known directory path
for file in os.listdir(known_dir):
  print(known_dir + '/' + file)
  img = read_img(known_dir + '/' + file) ##Reading all the images
  img_enc = face_recognition.face_encodings(img)[0] ##face encodings
  known_encodings.append(img_enc) 
  known_names.append(file.split('.')[0])
print(known_names) ## Printing all the known labels

#prepare encodings for unknown faces, 
#comparing them with known encoding and recognizing the faces
unknown_dir = 'unknown' ##UNknown Images Directory

imgList = os.listdir('/content/workspace/data_dst/aligned')
imgList.sort()

for file in imgList:
 print("Processing",file)

 if file != None:
  img = read_img('/content/workspace/data_dst/aligned' + '/' + file)##reading images
  #print(img)

  #if img[0] != None:
  img_enc = face_recognition.face_encodings(img)[0]##Encodings
  #print(img_enc)
  
  results = face_recognition.compare_faces(known_encodings,img_enc)
 
 res = [i for i, val in enumerate(results) if val]

 if len(res) > 0:
  print(res)
  name = known_names[res[0]]
  path = '/content/workspace/data_dst/aligned' + '/' + file
  if name != "hania":
    shutil.move(path, "/content/face2")
    print(file, "moved")


# Anvil Key

In [ ]:
import anvil.server
#pass your key as the function parameter
anvil.server.connect("RTGURUU3BMIXOYBUC7SJK5PV-UZNAAWUAFEUHU7GT")

# Anvil Functions

In [ ]:
#before you get input from user, you should have workspace already in here
import shutil
import os
from zipfile import ZipFile

#link for ws1: https://drive.google.com/file/d/1e0jF-jr-U-M0d38dfUU5XQMygBhIYVht/view?usp=sharing

@anvil.server.callable
def getDstVid(vidID, face):
  print("inside get DST")
  zipped = None

  if vidID == 1:
    !gdown --id 1e0jF-jr-U-M0d38dfUU5XQMygBhIYVht             #download ws1
    zipped = 'v1.zip'
  if vidID == 2 and face == 1:
    print("ws2f1 to be downloaded")
    !gdown --id 1Dcz2kW7P7JCnUfcdvMfScgtFKyIlP-jj
    zipped = 'v2f1.zip'
  if vidID == 2 and face == 2:
    !gdown --id                                               #download ws2
    zipped = 'v2f2.zip'

  with ZipFile(zipped, 'r') as zipObj:
    zipObj.extractall()
  
  #download model/base/merger
  #!gdown id --
  
  return 1  

#getDstVid(2, 1)


In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
#gdown only works for direct links
#if we get the link from frontend we need to save it in a variable
#gdown consideres the variables as the link instead of the data of the variable

import re
@anvil.server.callable
def getSrcLink(link):

  print("inside get SRC")
  pattern = "d/(.*?)/view"
  file_id = re.search(pattern, link).group(1)

  #video can be downloaded but for images we need zipped folder
  destination = '/content/workspace/data_src.mp4'
  download_file_from_google_drive(file_id, destination)

  return "Input Received"

#getSrcLink("https://drive.google.com/file/d/1MMnSxwBYzl5q8a91qQ5yWtP5vi3Hqg8P/view?usp=sharing")

In [ ]:
#sending mail through google

import anvil.google.mail
import anvil.google.auth

@anvil.server.callable
def send_mail(email_from,to,subject,body):
  email_addr = anvil.google.auth.get_user_email()
  print ("%s is now logged in" % email_addr)
  anvil.google.mail.send(from_address = email_from , to="Anvil Admin<l174557@lhr.nu.edu.pk>", subject="Request for New Destionation Vid", text = body)
  pass

# Import/Export from Drive

In [ ]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "kate.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")



In [ ]:
#@title Export to Drive { form-width: "30%" }
Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "merged_mask", "models", "result video", "result_mask video"]
Archive_name = "kate.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/workspace/data_dst", "merged")
elif Mode == "merged_mask":
  zip_and_copy("/content/workspace/data_dst", "merged_mask")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
elif Mode == "result video":
  !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/
  
print("Done!")


Mounted at /content/drive
/content
Done!


# Preprocess Input

In [ ]:
@anvil.server.callable
def preprocess():
  #@title Extract frames
  print("extracting frames")
  Video = "data_src" #@param ["data_src", "data_dst"]

  %cd "/content"
  cmd = "DeepFaceLab/main.py videoed extract-video "
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/ --output-ext jpg --fps 25"
    
  !python $cmd

  #@title Detect faces
  print("detecting faces")
  Data = "data_src" #@param ["data_src", "data_dst"]
  Detector = "S3FD" #@param ["S3FD", "S3FD (whole face)"]
  Debug = False #@param {type:"boolean"}

  detect_type = "s3fd"
  dbg = " --output-debug" if Debug else " --no-output-debug"

  folder = "workspace/"+Data
  folder_aligned = folder+"/aligned"

  cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
  cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+ dbg
  cmd+=" --face-type whole_face --image-size 512 --jpeg-quality 90 --max-faces-from-image 0"

  %cd "/content"
  !python $cmd

  preprocess()


In [ ]:
#@anvil.server.callable

def preprocess():
  #@title Extract frames
  print("extracting frames")
  Video = "data_dst" #@param ["data_src", "data_dst"]

  %cd "/content"
  cmd = "DeepFaceLab/main.py videoed extract-video "
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/ --output-ext jpg --fps 30"
    
  !python $cmd

  #@title Detect faces
  print("detecting faces")
  Data = "data_dst" #@param ["data_src", "data_dst"]
  Detector = "S3FD" #@param ["S3FD", "S3FD (whole face)"]
  Debug = False #@param {type:"boolean"}

  detect_type = "s3fd"
  dbg = " --output-debug" if Debug else " --no-output-debug"

  folder = "workspace/"+Data
  folder_aligned = folder+"/aligned"

  cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
  cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+ dbg
  cmd+=" --face-type whole_face --image-size 512 --jpeg-quality 90 --max-faces-from-image 0"

  %cd "/content"
  !python $cmd

preprocess()


extracting frames
/content
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --

In [ ]:
import shutil
shutil.make_archive("FILESRC", 'zip', '/content/workspace/data_dst')

'/content/FILESRC.zip'

# Train

In [ ]:
@anvil.server.background_task
def trainer():
  print("inside trainer")
  cmd = "DeepFaceLab/main.py train"
  cmd+= " --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain "
  cmd+= "--model-dir workspace/model --model SAEHD --force-gpu-idxs 0" 

  Silent_Start = False #@param {type:"boolean"}

  savedmodel = False
  if savedmodel == True:
    if Silent_Start:
      cmd+= " --silent-start"

  print("training started")

  train_cmd = (cmd)
  !python $train_cmd

  

@anvil.server.callable
def trainModel():
  """Launch a single background task."""
  task = anvil.server.launch_background_task('trainer')
  return task
  
@anvil.server.callable
def killTrainer(task):
  print("killing task")
  task.kill()
  return 0


In [ ]:
import cv2
import base64
  


@anvil.server.callable
def preview():

  file = '/content/workspace/model/New_SAEHD_preview_SAEHD.jpg'
  image = open(file, 'rb')
  image_read = image.read()
  image_64_encode = base64.encodebytes(image_read)

  image_binary = base64.b64decode(image_64_encode)
  my_media = anvil.BlobMedia(content_type="img/jpg", content=image_binary, name="acme.png")
  return my_media


# Merge Frames

In [ ]:
import base64

@anvil.server.callable
def merge():
  #@title Merge
  Model = "SAEHD" #@param ["SAEHD", "Quick96" ]

  cmd = "DeepFaceLab/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --force-gpu-idxs 0 --model-dir workspace/model --model "+Model

  %cd "/content"
  !python $cmd

  #@title Get result video 
  Mode = "result video" #@param ["result video", "result_mask video"]
  Copy_to_Drive = True #@param {type:"boolean"}


  if Mode == "result video":
    !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --bitrate 16 --include-audio 
    if Copy_to_Drive:
      !cp /content/workspace/result.mp4

  with open("/content/workspace/result.mp4", "rb") as videoFile:
    text = base64.b64encode(videoFile.read())
    vid = (base64.b64decode(text))

    my_media = anvil.BlobMedia(content_type="vid/mp4", content=vid, name="vid.mp4")
    return my_media

# Anvil Wait Forever

In [ ]:
anvil.server.wait_forever()